In [3]:
#!/usr/bin/env python3
#coding=utf-8
import time
from Arm_Lib import Arm_Device
from ikpy import chain
from ikpy import link
from ikpy.link import URDFLink #ik
from ikpy.chain import Chain #ik
import numpy as np

# Create robot arm object
Arm = Arm_Device()
time.sleep(.1) #tenth of a second delay

#Defining variables
angle = 90
s_time = 1000 #milliseconds
gripper = 6

In [4]:
def home(): #set arm to home position after task is complete. Useful for ik
    Arm.Arm_serial_servo_write6(angle, angle, angle, angle, angle, angle-90, s_time)

In [3]:
#Better method then below, sources link information straight from doc that holds the physical configuration of a robot model
urdf_file_path = '/home/jetson/dofbot_ws/src/dofbot_moveit/urdf/dofbot.urdf'
dof_chain = Chain.from_urdf_file(urdf_file_path)

/home/jetson/.local/lib/python3.6/site-packages/ikpy/chain.py:60: UserWarning: Link Base link (index: 0) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive
  warnings.warn("Link {} (index: {}) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive".format(link.name, link_index))


In [4]:
# # Defining a kinematic chain with 6 URDFLinks detailing the base and 5 moving links with their respective positions in meters, #origin_orientations, and rotation axes. Values are based off home function.
# dof_chain = chain.Chain(links=[
#     link.URDFLink(
#       name="base",
#       origin_translation=[0, 0, 0], #distance from previous joint to current joint in 3D space 
#       origin_orientation=[0, 0, 0], #rotation angles
#       rotation=[0, 0, 1], #capability of rotation, for base use z-axis to define the axis around which any potential rotation could occur if it were not fixed. The base itself does not actually rotate but is the reference point from which all other rotations are calculated.
#       bounds=[-1.5708, 1.5708] #radians
#     ),
#     link.URDFLink(
#       name="first_link",
#       origin_translation=[0, 0, 0.06605],
#       origin_orientation=[-0.010805, 0, 0],
#       rotation=[0, 0, 1],
#       bounds=[-1.5708, 1.5708]
#     ),
#     link.URDFLink(
#       name="second_link",
#       origin_translation=[0, -0.00031873, 0.04145], 
#       origin_orientation=[0, 1.5708, 0],
#       rotation=[0, 0, 1],
#       bounds=[-1.5708, 1.5708]
#     ),
#     link.URDFLink(
#       name="third_link",
#       origin_translation=[-0.08285, 0, 0], 
#       origin_orientation=[0, 0, 0],
#       rotation=[0, 0, 1],
#       bounds=[-1.5708, 1.5708]
#     ),
#     link.URDFLink(
#       name="fourth_link",
#       origin_translation=[-0.08285, 0, 0],  
#       origin_orientation=[0, 0, 0.0083081],
#       rotation=[0, 0, 1],
#       bounds=[-1.5708, 1.5708]
#     ),
#     link.URDFLink(
#       name="fifth_link",
#       origin_translation=[-0.07385, -0.001, 0], 
#       origin_orientation=[0, -1.57, 0],
#       rotation=[0, 0, 1],
#       bounds=[-1.5708, 3.1416]
#     ),
# ], active_links_mask=[False, True, True, True, True, True]) # This parameter is a list of booleans indicating whether a link is active (i.e., whether it should be considered during the inverse kinematics computation). For example, if you have a fixed base that should not move, you mark it as False. An active link (one that moves) is marked as True.

In [5]:
def home_to_cupstack():
    Arm.Arm_serial_servo_write6(angle-90, angle, angle, angle, angle, angle-90, s_time)
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-45, angle, angle, angle, angle-90, s_time)
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-45, angle-90, angle+45, angle, angle-90, s_time)
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-45, angle-90, angle+45, angle, angle+90, s_time) #gripper closing 
    time.sleep(1) 
    Arm.Arm_serial_servo_write6(angle-90, angle-30, angle-75, angle+25, angle, angle+90, s_time) #lifting cup straight up
    time.sleep(1)   

In [6]:
def cupstack_to_dispenser():
    # Define target position
    target_vector = [1, 1, -3] # Desired position of the gripper relative to the base frame, which is at [0, 0, 0]. This is the location where the gripper will end at and the joints will move accordingly. 
    frame_target = np.eye(4) # 4x4 matrix for Rot, Pos and an extra dimension for homogeneous coordinates
    frame_target[:3, 3] = target_vector
    position = frame_target[:3, 3]
    #do we need a line for defining the rotations of target frame corresponding to the #origin_orientation of the gripper? 
    
    #function within ikpy that takes the setup chain and does the math for ik to reach specified target
    ik = dof_chain.inverse_kinematics(position) # Compute the inverse kinematics

    # Translate IK angles to servo commands
    servo_angles = np.degrees(ik) # Convert radians to degrees
    print(servo_angles)
    Arm.Arm_serial_servo_write(1, servo_angles[0], s_time)
    time.sleep(1) #one sec delay
    Arm.Arm_serial_servo_write(2, servo_angles[1], s_time)
    time.sleep(1) #one sec delay
    Arm.Arm_serial_servo_write(3, servo_angles[2], s_time)
    time.sleep(1) #one sec delay
    Arm.Arm_serial_servo_write(4, servo_angles[3], s_time)
    time.sleep(1) #one sec delay
    Arm.Arm_serial_servo_write(5, servo_angles[4], s_time)
    time.sleep(1) #one sec delay
    Arm.Arm_serial_servo_write(6, 90, s_time)
#     Arm.Arm_serial_servo_write6(*servo_angles, s_time) # '*' because servo_angles holds a list of angles when storing convert_to_servo_commands(ik)
    
#     time.sleep(5) #five sec delay to allow for whole arm to move  

#     #Release cup
#     Arm.Arm_serial_servo_write(gripper, 0, s_time)
#     time.sleep(1)  
    

In [1]:
# import the needed modules (as is used to rename the module for easier referencing)
import math
import numpy as np
import general_robotics_toolbox as rox
import time
from Arm_Lib import Arm_Device

Arm = Arm_Device()

# define the Jacobian inverse 
def jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol):
# the inputs are 
	# robot: of the Robot class defined in rox. contains rotation axes, position vectors, joint types
	# q0: the initial guess joint angles as a 5x1 numpy array
	# Rd: the desired 3x3 rotation matrix (R_{0T}) as a numpy array
	# Pd: the desired 3x1 position vector (P_{0T}) as a numpy array
	# Nmax: the maximum number of allowed iterations
	# alpha: the update parameter
	# tol: the tolerances for [roll, pitch, yaw, x, y, z] as a 6x1 numpy array

# set up storage space
    n = len(q0)
    q = np.zeros((n,Nmax+1))
    q[:,0] = q0
    p0T = np.zeros((3,Nmax+1))
    RPY0T = np.zeros((3,Nmax+1))
    iternum = 0

# compute the forward kinematics
    H = rox.fwdkin(robot,q[:,0])
    R = H.R
    P = H.p
    P = np.array([[P[0]], [P[1]], [P[2]]])

# get the initial error
    dR = np.matmul(R, np.transpose(Rd))
    r = np.array(rox.R2rpy(dR))[None]
    dX = np.concatenate((np.transpose(r), P-Pd))

# iterate while any error element is greater than its tolerance
    while (np.absolute(dX) > tol).any():
	# stop execution if the maximum number of iterations is exceeded
        if iternum < Nmax:
		# compute the forward kinematics
            H = rox.fwdkin(robot, q[:,iternum])
            R = H.R
            p0T = H.p
            p0T = np.array([[p0T[0]], [p0T[1]], [p0T[2]]])
		# compute the error
            dR = np.matmul(R , np.transpose(Rd))
            r = np.array(rox.R2rpy(dR))[None]
            dX = np.concatenate((np.transpose(r), p0T-Pd))

		# calculate the Jacobian matrix
            Jq = rox.robotjacobian(robot, q[:, iternum])
		# compute the update
            j = np.matmul(np.linalg.pinv(Jq), dX)
		# use the update to generate a new q
            q[:, iternum+1] = q[:, iternum] - np.transpose((alpha * j))
            iternum = iternum + 1
        else:
            break
	# return the final estimate of q
    return q[:, iternum]

def main():

	#define inputs
	
    #qd=[30;45;80;25;40]
    #Rd = np.array([ [-0.75, -0.1047, -0.6531], [-0.433, 0.8241, 0.3652], [0.5, 0.5567, -0.6634]])
    #Pd = np.array([[0.2058], [0.1188], [0.1464]])
    #q0 = np.array(np.transpose([25, 50, 75, 30, 30]))*math.pi/180

    #qd=[0;45;135;45;135] desired position 
    Rd = np.array([ [-0.7071, -0.5, -0.5], [0, -0.7071, 0.7071], [-0.7071, 0.5, 0.5]])
    Pd = np.array([[0.1494], [0], [0.3367]])
	# convert initial guess to radians
    q0 = np.array(np.transpose([0, 0, 0, 0, 0]))*math.pi/180
    
    tol = np.array(np.transpose([[0.02, 0.02, 0.02, 0.001, 0.001, 0.001]]))
    Nmax = 200
    alpha = 0.1

    # Define all the joint lengths [m]
    l0 = 61 * 10**-3
    l1 = 43.5 * 10**-3
    l2 = 82.85 * 10**-3
    l3 = 82.85 * 10**-3
    l4 = 73.85 * 10**-3
    l5 = 54.57 * 10**-3

	# define the unit vectors
    ex = np.array([1, 0, 0])
    ey = np.array([0, 1, 0])
    ez = np.array([0, 0, 1])

    # Define the position vectors from i-1 -> i
    P01 = (l0 + l1) * ez
    P12 = np.zeros(3)
    P23 = l2 * ex
    P34 = -1*l3 * ez
    P45 = np.zeros(3)
    P5T = -1*(l4 + l5) * ex

	# define the class inputs: rotation axes (H), position vectors (P), and joint_type
    H = np.array([ez, -1*ey, -1*ey, -1*ey, -1*ex]).T
    P = np.array([P01, P12, P23, P34, P45, P5T]).T
    joint_type = [0,0,0,0,0]

	# define the Robot class
    robot = rox.Robot(H, P, joint_type)
	
	# compute the inverse kinematics
    q = jacobian_inverse(robot,q0,Rd,Pd,Nmax,alpha,tol)s
	# convert solution to degrees
    q = q * 180 / math.pi
    print(q) #this gives an array of q values for the end effector. Could be simply inputted to servo commands like we used for fk. 
    Arm.Arm_serial_servo_write(1, q[0], 1000)
    time.sleep(1) #one sec delay
#     Arm.Arm_serial_servo_write(2, q[0], 1000)
#     time.sleep(1) #one sec delay
#     Arm.Arm_serial_servo_write(3, q[1], 1000)
#     time.sleep(1) #one sec delay
#     Arm.Arm_serial_servo_write(4, q[2], 1000)
#     time.sleep(1) #one sec delay
#     Arm.Arm_serial_servo_write(5, q[3], 1000)
#     time.sleep(1) #one sec delay
#     Arm.Arm_serial_servo_write(6, q[4], 1000)

if __name__ == "__main__" :
    main()

[ 179.89767967  135.6238563    43.4953582  -224.25753181  -44.85146606]


In [6]:
def main():
    home()
    time.sleep(2) #one sec delay
#     home_to_cupstack() #(FK)
#     home()
#     time.sleep(1) 
#     cupstack_to_dispenser() #(IK)
#     home()
#     time.sleep(1)
#     push_to_dispense() #(IK) pushes button to dispense drink 
#     home()
#     time.sleep(1) 
#     your_drink() #(IK) after a certain amount of time the arm will grab the cup from under the dispenser and the arm will fancily show the user the cup it just poured  
    
try :
    main()
except KeyboardInterrupt:
     print(" Program closed! ")
     pass